In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue","mrpc")
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

2022-11-17 22:56:21.321751: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-17 22:56:21.485507: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-17 22:56:22.191178: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/anwer/anaconda3/lib/
2022-11-17 22:56:22.191240: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
def tokenize_function(examples):
    return tokenizer(examples['sentence1'],examples['sentence2'],truncation  = True)

In [3]:
tokenized_datasets = raw_datasets.map(tokenize_function,batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1","sentence2","idx"])
tokenized_datasets = tokenized_datasets.rename_column("label","labels")
tokenized_datasets.set_format("torch")


Loading cached processed dataset at /home/anwer/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-9c8984771bdb61f4.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

Loading cached processed dataset at /home/anwer/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-bb3182f47b66c823.arrow


In [4]:
data_collator = DataCollatorWithPadding(tokenizer)

In [6]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(tokenized_datasets["train"],shuffle=True,batch_size = 9, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_datasets["validation"],shuffle=True,batch_size = 9, collate_fn=data_collator)

In [9]:
for batch in train_dataloader:
    break
print({k:v.shape for k,v in batch.items()})

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([9]), 'input_ids': torch.Size([9, 77]), 'token_type_ids': torch.Size([9, 77]), 'attention_mask': torch.Size([9, 77])}


In [7]:
from transformers import AutoModelForSequenceClassification
checkpoint = "bert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels = 2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [16]:
outputs = model(**batch)

In [17]:
print(outputs.loss,outputs.logits.shape)

tensor(0.6248, grad_fn=<NllLossBackward0>) torch.Size([9, 2])


In [18]:
from transformers import AdamW
optimizer = AdamW(model.parameters(),lr = 5e-5)

In [19]:
loss = outputs.loss
loss.backward()
optimizer.step()
optimizer.zero_grad()


In [20]:
from transformers import get_scheduler

num_epochs = 3

num_training_steps = num_epochs * len(train_dataloader)

lr_scheduler = get_scheduler(
                "linear",
                optimizer = optimizer,
                num_warmup_steps=0,
                num_training_steps = num_training_steps)

### Training full loops




In [25]:
import torch

device = "cpu"
model.to(device)
device

'cpu'

In [26]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()

for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k,v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1224 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [28]:
from datasets import load_metric
metric = load_metric("glue","mrpc")

model.eval()

for batch in eval_dataloader:
    batch = {k: v.to(device) for k,v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    
    logits = outputs.logits
    predictions = torch.argmax(logits, dim =1)
    metric.add_batch(predictions = predictions,references = batch["labels"])
    
metric.compute()

{'accuracy': 0.6838235294117647, 'f1': 0.8122270742358079}